# Use HYPER_PERSNL_Template Customized Notebook Template

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
infer_datetime_format=True

import numpy as np
import random

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

random.seed(0)

now1 = datetime.now()

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'CUSTOMER_ACCOUNT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
account = sf_df.to_pandas()

In [6]:
table_name = 'CUSTOMER_TRANSACTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
transactions = sf_df.to_pandas()

In [7]:
transactions.columns

Index(['ACC_BBAN', 'ACC_IBAN', 'ACC_OPEN_DT', 'BALANCE', 'CUST_ID', 'EXPENSE',
       'EXPENSE_CATEGORY', 'BROWSER', 'DEVICE', 'DR_CR', 'LOGIN_TIME',
       'LOGOUT_TIME', 'TXN_DT', 'TXN_ID', 'TXN_AMOUNT', 'TXN_CODE',
       'TXN_REMARKS', 'TXN_TIME', 'TXN_TYPE'],
      dtype='object')

In [8]:
transactions.sort_values(by=['CUST_ID', 'ACC_BBAN', 'TXN_DT', 'TXN_TIME'], inplace=True) 

In [9]:
table_name = 'CUSTOMER_FINANCE_BREAKDOWN'

sf_df = my_session.sql("select * from {}".format(table_name))
financials = sf_df.to_pandas()

In [10]:
table_name = 'CUSTOMER_SUBSCRIPTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
subscriptions = sf_df.to_pandas()
subscriptions_orig = subscriptions.copy(deep=True)

In [11]:
table_name = 'CUSTOMER_TERM_DEPOSIT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
term_deposit = sf_df.to_pandas()

In [12]:
table_name = 'CUSTOMER_LOAN_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
loan = sf_df.to_pandas()

In [13]:
table_name = 'CUSTOMER_CREDITCARD_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
credit_card = sf_df.to_pandas()

In [15]:
x1 = list(set(transactions['CUST_ID']))

In [19]:
account

,CUST_ID,ACC_IBAN,ACC_BBAN,ACC_OPEN_DT,BALANCE,ACC_STATUS,ACC_TYPE
0,4185,GB90EMIC89482763544524,GDJY66571183273954,2001-03-22,$50.57,Active,Credit
1,291,GB83VYSJ52094207716710,OVWN34297715684777,2010-08-29,"$74,111.13",Closed,Current
2,7497,GB17OCSO08226852523762,DLFO12501444293678,2000-09-04,$669.14,Active,Loan
3,7497,GB26RRMJ41734703010562,SZPN32742192244415,2010-09-05,$0.03,Active,Credit
4,7497,GB13JBDZ14742645466984,DAGC95804976124339,2006-02-14,$0.71,Closed,Credit
...,...,...,...,...,...,...,...
3101,2137,GB79MEEM68164816755595,JGQZ63653199537639,2020-07-08,$191.20,Active,Savings
3102,2137,GB72FWKV94464237567600,LCGK26173403352193,2011-06-05,"$54,340.79",Closed,Savings
3103,3899,GB97XFCA41641435894700,IQWX43085486211593,2006-01-08,$906.56,Active,Loan
3104,3899,GB72HHTA49364267690196,PFVP16819625415314,2014-10-14,$17.61,Closed,Loan


In [22]:
#bank_name = list(set(account['BANK_NAME'])) --> Unable to find Bank_Name in account table.
bank_name = ['UBS',
'Bank of America',
'Discover Financial',
'Credit Suisse',
'Santander Bank',
'American Express',
'HSBC Bank USA',
'JPMorgan Chase',
'BankUnited',
'Citigroup',
'Barclays',
'RBC Bank',
'Citizens Financial Group',
'Capital One',
'Goldman Sachs',
'Wells Fargo',
'Morgan Stanley']

# Reversing a list using slicing technique

In [21]:
def Reverse(lst):
    new_lst = lst[::-1]
    return new_lst

# Initiate the DataFrames 

In [23]:
EOD_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'DT', 'EOD_BALANCE', 
                                  'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 'EOD_TOTAL_INCOME', 
                                  'EOD_TOTAL_EXPENSE'])

In [24]:
EOD_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'DT', 'EOD_BALANCE', 'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 
                                  'EOD_TOTAL_INCOME', 'EOD_TOTAL_BALANCE'])

In [25]:
EOW_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE', 'EOW_TOTAL_DR', 
                                        'EOW_TOTAL_CR','EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE', 'SLOPE_BALANCE',
                              'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [26]:
EOW_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE',
       'EOW_TOTAL_CR', 'EOW_TOTAL_DR', 'EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE','SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [27]:
EOM_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'MTH_NUM-YR', 'EOM_BALANCE',
       'EOM_TOTAL_DR', 'EOM_TOTAL_CR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [28]:
EOM_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'MTH_NUM-YR', 'EOM_BALANCE', 'EOM_TOTAL_CR', 
       'EOM_TOTAL_DR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE',
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [29]:
Cust_Dashboard = pd.DataFrame(columns = ['CUST_ID', 'BALANCE', 'AVG_BALANCE', 'AVG_INCOME', 'AVG_EXPENSE', 'BANK_NAME'])   

# Aggregation logic for Daily, Weekly and Monthly 

In [116]:
transactions['TXN_DT'] = pd.to_datetime(transactions['TXN_DT'], format='%m/%d/%Y')

In [136]:
for i in x1: ### This is coming from account table
    ibans = list(set(transactions[transactions['CUST_ID']==i]['ACC_IBAN']))
    for j in ibans:  ### for a given customer pickup one IBAN at a time
        txn_temp = transactions.loc[(transactions['CUST_ID']==i) & (transactions['ACC_IBAN']==j), ['ACC_BBAN', 'ACC_IBAN','BALANCE', 'TXN_DT', 'DR_CR', 'TXN_AMOUNT', 'TXN_CODE']]    ### This is at account level
        txn_Dr = txn_temp.loc[(txn_temp['DR_CR']=='Dr')]    ### This is at account level
        txn_Cr = txn_temp.loc[(txn_temp['DR_CR']=='Cr')]    ### This is at account level
        ##txn_temp['txn_Dt'] = txn_temp['txn_Dt'].astype(str)

        #### Process as per balance column for EOD Balance ##############
        #Dts = txn_temp['TXN_DT'].drop_duplicates() 
        Dts = pd.to_datetime(txn_temp['TXN_DT'].drop_duplicates(), format='%m/%d/%Y')
        
        sdate = datetime.strptime(str(min(Dts))[:10], '%Y-%m-%d')
        edate = datetime.strptime(str(max(Dts))[:10], '%Y-%m-%d')        
        
        dt_range = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
        hn = len(dt_range)
        eod_balance = []

        for di in Dts:
            eod_balance = eod_balance + [txn_temp.loc[txn_temp['TXN_DT']==di, ['TXN_DT', 'BALANCE']].iloc[-1, 1]]
        temp_Acc_Daily = pd.DataFrame({'DT': list(Dts), 'EOD_BALANCE': eod_balance})
        temp_Acc_Daily.reset_index(inplace=True) 
        temp_Acc_Daily['DT']=temp_Acc_Daily['DT'].astype('str')
        delta_Acc_Daily = pd.DataFrame({'CUST_ID': [i]*hn, 'ACC_IBAN': [j]*hn, 'ACC_BBAN':[txn_temp.iloc[0,0]]*hn, 
                                        'DT': dt_range})
        delta_Acc_Daily['DT']=delta_Acc_Daily['DT'].astype('str')
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, temp_Acc_Daily, on='DT', how='left')
        
        ## delta_Acc_Daily.to_csv('delta_Acc_Daily.csv', index=False)
        delta_Acc_Daily.drop('index', axis=1, inplace=True)
        gt0_values = delta_Acc_Daily.loc[delta_Acc_Daily['EOD_BALANCE']>0, 'EOD_BALANCE']
        gt0_idx = list(gt0_values.index) + [len(delta_Acc_Daily)]
        eod_balance=[]
        for idx in range(len(gt0_idx)-1):
            eod_balance = eod_balance+[gt0_values[gt0_idx[idx]]]*(gt0_idx[idx+1]-gt0_idx[idx])
        delta_Acc_Daily['EOD_BALANCE'] = eod_balance

        #### Process Debit amount as per dr_cr column ##############
        Dr_sum = pd.DataFrame(txn_Dr.groupby('TXN_DT')['TXN_AMOUNT'].sum())
        Dr_sum['TXN_DT'] = Dr_sum.index
        Dr_sum.index = range(len(Dr_sum))
        Dr_sum.rename(columns = {'TXN_DT':'DT', 'TXN_AMOUNT':'EOD_TOTAL_DR'}, inplace=True)

        #### Process Credit amount as per dr_cr column ##############
        Cr_sum = pd.DataFrame(txn_Cr.groupby('TXN_DT')['TXN_AMOUNT'].sum())
        Cr_sum['TXN_DT'] = Cr_sum.index
        Cr_sum.index = range(len(Cr_sum))
        Cr_sum.rename(columns = {'TXN_DT':'DT', 'TXN_AMOUNT':'EOD_TOTAL_CR'}, inplace=True)        
        
        ### Now merge the dataframes ###########
        Dr_sum['DT']=Dr_sum['DT'].astype('str')
        Cr_sum['DT']=Cr_sum['DT'].astype('str')
        
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, Dr_sum, on='DT', how='left')
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, Cr_sum, on='DT', how='left')
        
        delta_Acc_Daily['EOD_TOTAL_INCOME']=delta_Acc_Daily['EOD_TOTAL_CR']
        delta_Acc_Daily['EOD_TOTAL_EXPENSE']=delta_Acc_Daily['EOD_TOTAL_DR']               
        ##print(delta_Acc_Daily.shape)
        delta_Acc_Daily.fillna(0, inplace=True)

        #EOD_Acc_Level=EOD_Acc_Level.append(delta_Acc_Daily) --> Throwing error
        EOD_Acc_Level = pd.concat([EOD_Acc_Level, delta_Acc_Daily], ignore_index=True)
        

        #### Process for EOW Balance ##############        
        temp_Acc_Wkly = delta_Acc_Daily
        temp_Acc_Wkly['WK_NUM-YR'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-W%W') for x in temp_Acc_Wkly['DT']]
        temp_Acc_Wkly.replace(np.nan, 0, inplace=True)
        temp_Dt = [datetime.strptime(x, '%Y-%m-%d') for x in temp_Acc_Wkly['DT']]
        temp_Acc_Wkly['WK_START_DT'] = [x - timedelta(days=x.weekday()) for x in temp_Dt]
        temp_Acc_Wkly['WK_END_DT'] = [x + timedelta(days=6) for x in temp_Acc_Wkly['WK_START_DT']]

        ##transactions.loc[(transactions['Cust_ID']==i) & (transactions['Acc_IBAN']==j), ['Acc_BBAN', 'Balance', 'txn_Dt', 'dr_cr', 'txn_amount', 'txn_code']]    ### This is at account level
        
        ##temp = temp_Acc_Wkly.groupby('Wk_Num-Yr')[['EOD_Total_Dr', 'EOD_Total_Cr', 'EOD_Total_Income', 'EOD_Total_Expense']].sum()
        
        ##delta_Acc_Wkly = pd.DataFrame(temp_Acc_Wkly.groupby('Wk_Num-Yr').agg({'EOD_Balance':'last', 'EOD_Total_Dr':'sum', 'EOD_Total_Cr':'sum', 'EOD_Total_Income':'sum', 'EOD_Total_Expense':'sum'}))
        
        delta_Acc_Wkly = pd.DataFrame(temp_Acc_Wkly.groupby(['CUST_ID', 'ACC_IBAN', 'ACC_BBAN','WK_NUM-YR', 'WK_START_DT', 'WK_END_DT']).agg({'EOD_BALANCE':'last', 'EOD_TOTAL_DR':'sum', 'EOD_TOTAL_CR':'sum', 'EOD_TOTAL_INCOME':'sum', 'EOD_TOTAL_EXPENSE':'sum'}))
        delta_Acc_Wkly.reset_index(inplace=True)
                
        delta_Acc_Wkly.replace(np.nan, 0, inplace=True)
        ##temp_Dt = [datetime.strptime(x, '%Y-%m-%d') for x in temp_Acc_Wkly['Dt']]
        ##delta_Acc_Wkly['Wk_Start_Dt'] = set([x - timedelta(days=x.weekday()) for x in temp_Dt])
        ##delta_Acc_Wkly['Wk_End_Dt'] = set([x + timedelta(days=6) for x in delta_Acc_Wkly['Wk_Start_Dt']])

        #### Find the slope for accounts at daily level for each block of Wk_Num-Yr ######
        slope_cols = ['SLOPE_BALANECE', 'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME', 'SLOPE_TOTAL_EXPENSE']        
        amount_cols = ['EOD_BALANCE', 'EOD_TOTAL_DR', 'EOD_TOTAL_CR', 'EOD_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE']

        for iamt in range(len(amount_cols)):
            slope_temp = []
            for iwk in sorted(list(set(temp_Acc_Wkly['WK_NUM-YR']))):
                slope_EOD_Bal = list(temp_Acc_Wkly[amount_cols[iamt]].loc[(temp_Acc_Wkly['CUST_ID']==i) & (temp_Acc_Wkly['ACC_IBAN']==j) & (temp_Acc_Wkly['WK_NUM-YR']==iwk)])
                #if any([len(slope_EOD_Bal)<2]+[math.isnan(x) for x in slope_EOD_Bal]):
                if len(slope_EOD_Bal)<2:
                    slope_temp = slope_temp + [0]
                else:
                    slope_temp = slope_temp + [round(np.polyfit(list(range(len(slope_EOD_Bal))), slope_EOD_Bal, 1)[0], 2)]
            delta_Acc_Wkly[slope_cols[iamt]]=slope_temp

        
        ##r = datetime.datetime.strptime(temp_Acc_Wkly['Wk_Num-Yr'][0] + '-1', '%G-W%V-%u')
        delta_Acc_Wkly.rename(columns = {'EOD_BALANCE':'EOW_Balance', 'EOD_Total_Dr':'EOW_Total_Dr', 'EOD_Total_Cr':'EOW_Total_Cr', 'EOD_Total_Income':'EOW_Total_Income', 'EOD_Total_Expense':'EOW_Total_Expense'}, inplace=True)
        delta_Acc_Wkly.fillna(0, inplace=True)
        EOW_Acc_Level = EOW_Acc_Level.append(delta_Acc_Wkly)
    

In [137]:
delta_Acc_Wkly

,CUST_ID,ACC_IBAN,ACC_BBAN,WK_NUM-YR,WK_START_DT,WK_END_DT,EOD_BALANCE,EOD_TOTAL_DR,EOD_TOTAL_CR,EOD_TOTAL_INCOME,EOD_TOTAL_EXPENSE,SLOPE_BALANECE,SLOPE_TOTAL_CR,SLOPE_TOTAL_DR,SLOPE_TOTAL_INCOME,SLOPE_TOTAL_EXPENSE
0,100002,GB91DITZ74834780737967,CUHL01704932377151,2017-W15,2017-04-10,2017-04-16,4681.74,1334.64,1334.64,1334.64,1334.64,-0.00,-1334.64,-1334.64,-1334.64,-1334.64
1,100002,GB91DITZ74834780737967,CUHL01704932377151,2017-W16,2017-04-17,2017-04-23,4828.79,294.10,147.05,147.05,294.10,15.76,31.51,15.76,15.76,31.51
2,100002,GB91DITZ74834780737967,CUHL01704932377151,2017-W17,2017-04-24,2017-04-30,3185.87,1348.82,0.00,0.00,1348.82,-176.03,-96.34,0.00,0.00,-96.34
3,100002,GB91DITZ74834780737967,CUHL01704932377151,2017-W18,2017-05-01,2017-05-07,2366.14,2459.19,819.73,819.73,2459.19,-175.66,87.83,29.28,29.28,87.83
4,100002,GB91DITZ74834780737967,CUHL01704932377151,2017-W19,2017-05-08,2017-05-14,2366.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,100002,GB91DITZ74834780737967,CUHL01704932377151,2021-W36,2021-09-06,2021-09-12,23856.49,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00
234,100002,GB91DITZ74834780737967,CUHL01704932377151,2021-W37,2021-09-13,2021-09-19,25679.35,4988.51,9847.09,9847.09,4988.51,-60.24,-45.74,423.48,423.48,-45.74
235,100002,GB91DITZ74834780737967,CUHL01704932377151,2021-W38,2021-09-20,2021-09-26,29314.97,1332.23,3996.69,3996.69,1332.23,389.53,-95.16,-285.48,-285.48,-95.16
236,100002,GB91DITZ74834780737967,CUHL01704932377151,2021-W39,2021-09-27,2021-10-03,29314.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
